# VO Pipeline
Vision Algorithms for Mobile Robotics | Fall 2025 <br>
David Jensen, Alessandro Pirini, Matteo Rubini

## Parameters

In [3]:
# Dataset -> 0: KITTI, 1: Malaga, 2: Parking, 3: Own Dataset
ds = 2

## Imports

### Libraries

In [5]:
import os
from glob import glob

import cv2
import skimage

import numpy as np

import matplotlib as plt

### Data
_Ensure that all datasets have been downloaded and unzipped into their respective folders_

In [ ]:
# Define dataset paths
# (Set these variables before running)
kitti_path = "./kitti"
malaga_path = "./malaga"
parking_path = "./parking"
# own_dataset_path = "/path/to/own_dataset"

if ds == 0:
    assert 'kitti_path' in locals(), "You must define kitti_path"
    ground_truth = np.loadtxt(os.path.join(kitti_path, 'poses', '05.txt'))
    ground_truth = ground_truth[:, [-9, -1]]  # same as MATLAB(:, [end-8 end])
    last_frame = 4540
    K = np.array([
        [7.18856e+02, 0, 6.071928e+02],
        [0, 7.18856e+02, 1.852157e+02],
        [0, 0, 1]
    ])
elif ds == 1:
    assert 'malaga_path' in locals(), "You must define malaga_path"
    img_dir = os.path.join(malaga_path, 'malaga-urban-dataset-extract-07_rectified_800x600_Images')
    left_images = sorted(glob(os.path.join(img_dir, '*.png')))
    last_frame = len(left_images)
    K = np.array([
        [621.18428, 0, 404.0076],
        [0, 621.18428, 309.05989],
        [0, 0, 1]
    ])
elif ds == 2:
    assert 'parking_path' in locals(), "You must define parking_path"
    last_frame = 598
    K = np.loadtxt(os.path.join(parking_path, 'K.txt'))
    ground_truth = np.loadtxt(os.path.join(parking_path, 'poses.txt'))
    ground_truth = ground_truth[:, [-9, -1]]
elif ds == 3:
    # Own Dataset
    # TODO: define your own dataset and load K obtained from calibration of own camera
    assert 'own_dataset_path' in locals(), "You must define own_dataset_path"

else:
    raise ValueError("Invalid dataset index")

# Initialization
- Select two keyframes with large enough baseline
- Use indirect (feature-based) or direct (KLT) method to establish keypoint corrispondences between frames
- Estimate relative pose and triangulate points to bootstrap point cloud (5-pt RANSAC)
- Initialize VO pipeline with inlier keypoints and their associated landmarks

# Operation
- Match keypoints in current image to existing landmarks
    - Extract keypoints (Harris)
    - Track (KLT)
- Estimate pose
    - Estimate pose and handle outliers (P3P plus RANSAC)
- Add new landmarks as needed by triangulating new features
    - Keep track of candidate landmarks
        - Keypoint itself
        - Observation when first seen
        - Pose when first seen
    - Only add when they have been tracked for long enough and baselineis large enough
    - Discard if track fails
